In [ ]:
# 런타임 리셋 : 세션 초기화됩니다. 주의!
import os
os.kill(os.getpid(), 9)

## README



```Markdown
> 테스트 모델 :
    - naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B

> 방식
    - LoRA

> 허깅페이스 아이디
    - haebo

```



## Setting

In [7]:
!pip install -U datasets

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from google.colab import userdata

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
HF_TOKEN = userdata.get('HF_TOKEN')
print(f'device : {DEVICE}')
print(f'HF_TOKEN : {True if HF_TOKEN else False}')
print(f"GPU : {torch.cuda.get_device_name(0)}")

device : cuda
HF_TOKEN : True
GPU : Tesla T4


In [9]:
model_id = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B"
print(f'model : {model_id}')
tokenizer = AutoTokenizer.from_pretrained(model_id, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto", # 모델 연산 CPU 분산
    token=HF_TOKEN
)

model : naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B


In [10]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(110592, 2048, padding_idx=100257)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=7168, bias=False)
          (up_proj): Linear(in_features=2048, out_features=7168, bias=False)
          (down_proj): Linear(in_features=7168, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e

In [11]:
!nvidia-smi
!free -h

Wed May 21 06:42:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   65C    P0             28W /   70W |    7036MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [12]:
from huggingface_hub import HfApi, login

# 1. 로그인 (토큰 파일에 저장)
login(token=HF_TOKEN)

# 2. API 객체로 로그인 상태 확인
api = HfApi()
print("✅ 현재 사용자:", api.whoami()["name"])

✅ 현재 사용자: haebo


## LoRA

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
import torch

# ------------------------------------------------------
# ✅ LoRA 설정
# ------------------------------------------------------
peft_config = LoraConfig(
    r=8,                                  # Low-rank rank
    lora_alpha=16,                        # Scaling factor
    target_modules=["q_proj", "v_proj"],  # add LoRA module : LLM에 따라 다를 수 있음
    lora_dropout=0.05,                    # Dropout on LoRA path
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    base_model_name_or_path="naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B"
)

model = get_peft_model(model, peft_config) # (중요) LoRA adapter 외 Freeze 및 adpter 활성화

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [16]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): LlamaForCausalLM(
          (model): LlamaModel(
            (embed_tokens): Embedding(110592, 2048, padding_idx=100257)
            (layers): ModuleList(
              (0-23): 24 x LlamaDecoderLayer(
                (self_attn): LlamaAttention(
                  (q_proj): lora.Linear(
                    (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=2048, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=2048, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
 

In [17]:
# 데이터셋 업데이트 설치 반드시 진행한 해야 오류 없음
from datasets import load_dataset, Features, Value
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
import shutil
import os

# 1. 파일 경로
file_path = "./posts-0515.jsonl"

# 2. 데이터 구조 명시 (features)
features = Features({
    "content": Value("string"),
    "emotion": Value("string"),
    "post_type": Value("string"),
    "transformed_content": Value("string")
})

# 3. 임시 캐시 디렉토리 설정 (삭제 예정)
temp_cache_dir = "./tmp_cache"
os.makedirs(temp_cache_dir, exist_ok=True)

# 4. 데이터셋 로드
dataset = load_dataset(
    "json",
    data_files=file_path,
    features=features,
    split="train",
    cache_dir=temp_cache_dir,
    verification_mode="no_checks"  # ⚠️ ignore_verifications는 deprecated

)

# 5. 캐시 디렉터리 삭제 (완전한 캐시 제거)
shutil.rmtree(temp_cache_dir, ignore_errors=True)

# ✅ 결과 확인
print(dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

{'content': 's3 이미지 url 잘 변경됐나', 'emotion': 'happy', 'post_type': 'dog', 'transformed_content': '어흥! 🐶 s3 이미지 url, 완전 잘 변경됐는지 궁금해! 내가 열심히 지켜볼게! 💖 혹시 문제 생기면 바로 짖을게! 멍! ☀️'}


In [18]:
def format_and_tokenize(example):
    text = (
        f"### 사용자:\n{example['content']}\n"
        f"### 감정:\n{example['emotion']}\n"
        f"### 동물:\n{example['post_type']}\n\n"
        f"### 말투 변환 결과:\n{example['transformed_content']}"
    )
    tokenized = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(format_and_tokenize, remove_columns=dataset.column_names)
tokenized_dataset

Map:   0%|          | 0/342 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 342
})

In [19]:
print(next(model.parameters()).device)
print(next(model.parameters()).dtype)

cuda:0
torch.float16


In [20]:
from transformers import TrainingArguments, Trainer
import torch

# 출력 경로 및 Hub 업로드 설정
output_dir = "./finetuned-ktb"
repo_name = "Meow-HyperCLOVAX-1.5B-LoRA-fp16"
repo_id = f"haebo/{repo_name}"

# ======================================================
# ✅ TrainingArguments 설정
# ======================================================
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=5,
    bf16=False,                          # 또는 fp16=True (GPU 환경에 따라 선택)
    fp16= True,
    logging_steps=1,
    save_strategy="epoch",
    report_to="none",
    remove_unused_columns=False,       # ⚠️ LoRA에서 꼭 필요
    push_to_hub=True,
    hub_model_id=repo_id,
    hub_strategy="end"
)

# ======================================================
# ✅ Trainer 구성
# ======================================================
trainer = Trainer(
    model=model,                        # LoRA 적용된 model
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=lambda data: { # torch.stack -> torch.tensor
        "input_ids": torch.tensor([f["input_ids"] for f in data]),
        "attention_mask": torch.tensor([f["attention_mask"] for f in data]),
        "labels": torch.tensor([f["labels"] for f in data])
    }
)

# ======================================================
# ✅ 학습 실행
# ======================================================
trainer.train()


<ipython-input-20-295b027ba58d>:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
1,11.862800
2,12.256000
3,11.903100
4,11.980500
5,11.860200
6,11.981800
7,12.166700
8,11.701500
9,11.900700
10,11.924400


TrainOutput(global_step=210, training_loss=10.23671414965675, metrics={'train_runtime': 402.306, 'train_samples_per_second': 4.25, 'train_steps_per_second': 0.522, 'total_flos': 6987696555687936.0, 'train_loss': 10.23671414965675, 'epoch': 4.888888888888889})

In [24]:
# ======================================================
# ✅ 모델 저장 (로컬)
# ======================================================
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

No files have been modified since last commit. Skipping to prevent empty commit.


('./finetuned-ktb/tokenizer_config.json',
 './finetuned-ktb/special_tokens_map.json',
 './finetuned-ktb/vocab.json',
 './finetuned-ktb/merges.txt',
 './finetuned-ktb/added_tokens.json',
 './finetuned-ktb/tokenizer.json')

In [25]:
print(f"[DEBUG] hub_model_id in trainer: {trainer.args.hub_model_id}")

[DEBUG] hub_model_id in trainer: haebo/Meow-HyperCLOVAX-1.5B-LoRA-fp16


In [27]:
# ======================================================
# ✅ Hugging Face Hub에 업로드 (Model Hub 전용)
# ======================================================
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

print(f"✅ 모델 업로드 완료: https://huggingface.co/{repo_id}")

Loading adapter weights from ./finetuned-ktb led to unexpected keys not found in the model: base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight, base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight, base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight, base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight, base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight, base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight, base_model.model.model.layers.1.self_attn.v_proj.lora_A.default.weight, base_model.model.model.layers.1.self_attn.v_proj.lora_B.default.weight, base_model.model.model.layers.10.self_attn.q_proj.lora_A.default.weight, base_model.model.model.layers.10.self_attn.q_proj.lora_B.default.weight, base_model.model.model.layers.10.self_attn.v_proj.lora_A.default.weight, base_model.model.model.layers.10.self_attn.v_proj.lora_B.default.weight, base_model.model.model.layers.11.self_at

README.md:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/5.52M [00:00<?, ?B/s]

✅ 모델 업로드 완료: https://huggingface.co/haebo/Meow-HyperCLOVAX-1.5B-LoRA-fp16


In [28]:
!nvidia-smi
!free -h

Wed May 21 07:04:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   68C    P0             28W /   70W |    8094MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----